In [1]:
import os
import requests
from bs4 import BeautifulSoup
from flask import Flask, request, abort
from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.messaging import Configuration, ApiClient, MessagingApi, ReplyMessageRequest, TextMessage
from linebot.v3.webhooks import MessageEvent, TextMessageContent

line_access_token = os.environ.get('LINE_ACCESS_TOKEN')
line_secret = os.environ.get('LINE_SECRET')
port = 5000

configuration = Configuration(access_token=line_access_token)
handler = WebhookHandler(line_secret)


In [2]:
def get_latest_article():
    url = 'https://ccc.technews.tw/'
    response = requests.get(url)
    if response.status_code != 200:
        return "無法存取該網址。"
    else:
        soup = BeautifulSoup(response.text, 'html.parser')
        latest_article = soup.find('div', class_='content')
        if latest_article:
            title = latest_article.find('h1', class_='entry-title').text.strip()
            link = latest_article.find('a')['href']
            images = latest_article.find_all('img')
            if images:
                last_image = images[-1]['src']
                return f"最新文章標題：{title}\n最新文章連結：{link}\n最後一張圖片：{last_image}"
            else:
                return f"最新文章標題：{title}\n最新文章連結：{link}\n該文章沒有圖片。"
        else:
            return "找不到最新文章。"



In [3]:
latest_info = get_latest_article()
print(latest_info)

最新文章標題：Galaxy Ring 價格曝光，傳聞與 Galaxy Watch 定價差不多
最新文章連結：https://ccc.technews.tw/2024/05/24/samsung-galaxy-ring-price-revealed/
最後一張圖片：https://img.technews.tw/wp-content/uploads/2024/02/29161928/Galaxy-Ring1-300x225.jpg


In [8]:
app = Flask(__name__)

@app.route("/", methods=['POST'])
def callback():
    signature = request.headers.get('X-Line-Signature', '')
    body = request.get_data(as_text=True)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    text = event.message.text
    if text.lower() == "latest":
        latest_info = get_latest_article()
        with ApiClient(configuration) as api_client:
            line_bot_api = MessagingApi(api_client)
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=latest_info)]
                )
            )

if __name__ == "__main__":
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


127.0.0.1 - - [24/May/2024 12:13:00] "POST / HTTP/1.1" 200 -
